# Data Analysis and Cleaning
analysis data and do some dirty work for cleaning
- <a href='#1'>1. analysis data</a> 
- <a href='#2'>2. task2</a> 
- <a href='#3'>3. task3</a>

In [1]:
import sys
import os 

import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

sys.path.append('../')
import conf

In [2]:
# global settings
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns',1000)
pd.set_option('display.width',100)
sns.set(rc={'figure.figsize':(11,4)})

In [1]:
# global variables
DEFAULT_MISSING_VALUE = 0
FONT = fm.FontProperties(fname = os.path.join(conf.LIB_DIR,'simsun.ttc'))

NameError: name 'fm' is not defined

In [4]:
# functions
def __dummy():
    pass

def diivide_files_into_raw_and_normalized():
    pass

### <a id='1'> 1.task1</a>

In [5]:
# disk_sample_fault_tag
fault_tag_df = pd.read_csv(os.path.join(conf.DATA_DIR,'disk_sample_fault_tag.csv'))

In [6]:
fault_tag_df.head()

,manufacturer,model,serial_number,fault_time,tag
0,A,1,disk_100102,2017-09-29,0
1,A,1,disk_100584,2018-05-02,6
2,A,1,disk_100669,2018-06-22,3
3,A,1,disk_101190,2018-02-19,5
4,A,1,disk_103158,2017-10-20,0


In [7]:
# disk_sample_smart_log_201707.csv
log_1711_norm_df = pd.DataFrame()
log_1711_raw_df = pd.DataFrame()

In [8]:
primary_key = ['serial_number','manufacturer', 'model', 'dt']
norm_cols = primary_key + ['smart_%s_normalized'%i for i in range(1,256)]
raw_cols = primary_key + ['smart_%sraw'%i for i in range(1,256)]

In [9]:
for sub_df in pd.read_csv(os.path.join(conf.DATA_DIR, 'disk_sample_smart_log_201707.csv'),chunksize=1e+4):
        log_1711_norm_df = pd.concat([log_1711_norm_df, sub_df[norm_cols]])
        log_1711_raw_df = pd.concat([log_1711_raw_df, sub_df[raw_cols]])

In [11]:
# remove nan column
log_1711_norm_df.dropna(axis=1, how='all', inplace=True)
log_1711_raw_df.dropna(axis=1,how='all', inplace=True)

In [12]:
log_1711_norm_df.to_csv(os.path.join(conf.DATA_DIR, 'disk_sample_smart_log_201707_norm.csv'),index=False)
log_1711_raw_df.to_csv(os.path.join(conf.DATA_DIR, 'disk_sample_smart_log_201707_raw.csv'),index=False)

In [13]:
log_1711_norm_df.head()

,serial_number,manufacturer,model,dt,smart_1_normalized,smart_3_normalized,smart_4_normalized,smart_5_normalized,smart_7_normalized,smart_9_normalized,smart_10_normalized,smart_12_normalized,smart_184_normalized,smart_187_normalized,smart_188_normalized,smart_189_normalized,smart_190_normalized,smart_191_normalized,smart_192_normalized,smart_193_normalized,smart_194_normalized,smart_195_normalized,smart_197_normalized,smart_198_normalized,smart_199_normalized,smart_240_normalized,smart_241_normalized,smart_242_normalized
0,disk_69207,A,1,20170711,78.0,95.0,100.0,100.0,91.0,82.0,100.0,100.0,100.0,100.0,100.0,100.0,72.0,100.0,100.0,100.0,28.0,18.0,100.0,100.0,200.0,NaN,NaN,NaN
1,disk_69207,A,1,20170712,78.0,95.0,100.0,100.0,91.0,82.0,100.0,100.0,100.0,100.0,100.0,100.0,72.0,100.0,100.0,100.0,28.0,18.0,100.0,100.0,200.0,NaN,NaN,NaN
2,disk_69207,A,1,20170718,78.0,95.0,100.0,100.0,91.0,81.0,100.0,100.0,100.0,100.0,100.0,100.0,71.0,100.0,100.0,100.0,29.0,18.0,100.0,100.0,200.0,NaN,NaN,NaN
3,disk_6922,A,1,20170711,81.0,96.0,100.0,100.0,87.0,88.0,100.0,100.0,100.0,100.0,100.0,100.0,76.0,100.0,100.0,100.0,24.0,57.0,100.0,100.0,200.0,NaN,NaN,NaN
4,disk_6922,A,1,20170709,81.0,96.0,100.0,100.0,87.0,88.0,100.0,100.0,100.0,100.0,100.0,100.0,76.0,100.0,100.0,100.0,24.0,54.0,100.0,100.0,200.0,NaN,NaN,NaN
